In [22]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import os

In [6]:
os.chdir("../..")

In [7]:
steam_df = pd.read_csv("data/raw/games.csv")
steam_df["total_reviews"] = steam_df["Positive"] + steam_df["Negative"]
steam_df["success"] = steam_df["total_reviews"] > 500
steam_success = steam_df[steam_df["success"] == 1]
steam_failure = steam_df[steam_df["success"] == 0]
steam_success_about = steam_success["About the game"]
steam_failure_about = steam_failure["About the game"]
steam_success_about.dropna(inplace=True)
steam_failure_about.dropna(inplace=True)

/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_36807/2350337761.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_success_about.dropna(inplace=True)
/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_36807/2350337761.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  steam_failure_about.dropna(inplace=True)


In [8]:
from transformers import pipeline

# # Allocate a pipeline for sentiment-analysis
# classifier = pipeline('sentiment-analysis')
# classifier('We are very sad to introduce pipeline to the transformers repository.')

In [9]:
pipe = pipeline(
    "text-classification", model="SamLowe/roberta-base-go_emotions", top_k=None
)

In [21]:
list(steam_success_about.iloc[:5])

TypeError: len() takes exactly one argument (0 given)

In [23]:
# Load the model and tokenizer
model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [109]:
# Load the model and tokenizer
model_name = "SamLowe/roberta-base-go_emotions"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


# Define a function to split text into chunks
def split_into_chunks(text, max_length=512):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = tokens["input_ids"][0]
    chunk_size = max_length - 2  # accounting for special tokens
    chunks = [
        input_ids[i : i + chunk_size] for i in range(0, len(input_ids), chunk_size)
    ]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


# Apply splitting to your texts
texts = steam_success_about.iloc[:1000]
chunked_texts = [chunk for text in texts for chunk in split_into_chunks(text)]

In [124]:
chunked_texts[999]

"Clean up the world with style! As an acrobatic janitor, you are an adept force against dust and disorder. Leap and dash off walls and ceilings, and deftly traverse precarious environments. Cleanse each level swiftly and thoroughly to achieve mastery in this 2D sweep-'em-up platformer. Key Features: Over 75 levels varying in difficulty from elegant exploration levels to intense challenges for hardcore players. Over 100 community-made levels included in their own area of the game. Compete on the online leaderboard and instantly watch replays of every players' best attempts. Play against your friends with local multiplayer matches. Roam through a myriad of beautiful environments accompanied by an original soundtrack. Make your own worlds with the level editor, and seamlessly share and play them with the rest of the community."

In [120]:
results = pipe(chunked_texts[:999])

IndexError: index out of range in self

In [115]:
scores_df = pd.DataFrame(
    index=pd.DataFrame(results[0]).sort_values(by="label")["label"]
)

In [116]:
for n in range(len(results)):
    scores_df[f"text_n_{n}_score"] = (
        pd.DataFrame(results[n]).sort_values(by="label").set_index("label")["score"]
    )

In [117]:
scores_df.T.median().sort_values(ascending=False)

label
neutral           0.603642
approval          0.149244
realization       0.014995
optimism          0.013813
admiration        0.011415
annoyance         0.006733
excitement        0.005002
caring            0.004440
joy               0.003644
desire            0.003629
disapproval       0.003395
curiosity         0.002802
confusion         0.002261
disappointment    0.002250
relief            0.001967
pride             0.001492
love              0.001412
amusement         0.001285
fear              0.001277
sadness           0.001265
anger             0.001252
surprise          0.001196
disgust           0.000955
gratitude         0.000891
nervousness       0.000788
embarrassment     0.000416
grief             0.000364
remorse           0.000318
dtype: float64

In [130]:
# Define a function to split text into chunks
def split_into_chunks(text, max_length=512):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = tokens["input_ids"][0]
    chunk_size = max_length - 2  # accounting for special tokens
    chunks = [
        input_ids[i : i + chunk_size] for i in range(0, len(input_ids), chunk_size)
    ]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]


# Apply splitting to your texts
texts = steam_failure_about.iloc[:500]
chunked_texts_failure = [chunk for text in texts for chunk in split_into_chunks(text)]

In [131]:
results = pipe(chunked_texts_failure[:500])

In [132]:
scores_df_failure = pd.DataFrame(
    index=pd.DataFrame(results[0]).sort_values(by="label")["label"]
)

In [133]:
for n in range(len(results)):
    scores_df_failure[f"text_n_{n}_score"] = (
        pd.DataFrame(results[n]).sort_values(by="label").set_index("label")["score"]
    )

/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_36807/4016241334.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scores_df_failure[f"text_n_{n}_score"] = pd.DataFrame(results[n]).sort_values(by='label').set_index('label')['score']
/var/folders/25/8m9ycrtx2kv84b8szslr4jfc0000gp/T/ipykernel_36807/4016241334.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  scores_df_failure[f"text_n_{n}_score"] = pd.DataFrame(results[n]).sort_values(by='label').set_index('label')['score']
/var/folders/25/8m9ycrtx2kv84b8szslr4jfc00

In [134]:
scores_df_failure.T.median().sort_values(ascending=False)

label
neutral           0.551908
approval          0.112985
optimism          0.016395
realization       0.013306
admiration        0.010448
excitement        0.007797
annoyance         0.006179
caring            0.005850
joy               0.004220
desire            0.004112
disapproval       0.003749
confusion         0.002818
curiosity         0.002773
disappointment    0.002127
relief            0.001810
love              0.001658
amusement         0.001510
surprise          0.001332
fear              0.001331
anger             0.001291
sadness           0.001183
pride             0.001164
disgust           0.000948
nervousness       0.000841
gratitude         0.000672
embarrassment     0.000435
grief             0.000380
remorse           0.000312
dtype: float64